In [29]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_colwidth', None)

In [30]:
purchased = pd.read_csv('csv/gf12-purchased.csv')
purchased = purchased[['Part-Num', 'Description', 'PK QTY', 'Locations']] # Only keep relevant columns
purchased['Part-Num'] = purchased['Part-Num'].str.replace('\t', '') # get rid of \t tab characters in Part-Num

po = pd.read_csv('csv/gf12-all-purchase-orders.csv')
po = po[['Type', 'Date', 'Num', 'Source-Name', 'Item', 'Qty', 'Cost-Price', 'Item-Description']] # Only keep relevant columns
po['Cost-Price'] = po['Cost-Price'].round(2) # Round Cost-Price to 2 decimal places
po['Part-Num'] = po['Item'].str.split('(').str[0] # Extract part number from item column
po['Part-Num'] = po['Part-Num'].str.strip() # Get rid of trailing whitespace
po['Date'] = pd.to_datetime(po['Date'], format='%m/%d/%y')
# po = po.groupby(['Type', 'Date', 'Num', 'Source-Name', 'Part-Num', 'Cost-Price', 'Item-Description'])[['Qty']].sum().reset_index() # Group by part number, item description, and cost price, and sum the quantity
po['PK QTY'] = po['Part-Num'].map(purchased.set_index('Part-Num')['PK QTY']) # Map PK QTY from purchased to po
po['PK QTY'] = po['PK QTY'].fillna(1) # If PK QTY is NaN, set to 1
po['Unit-Qty'] = po['Qty'] * po['PK QTY'] # Multiply Qty by PK QTY
po['Unit-Price'] = po['Cost-Price'] / po['PK QTY'] # Calculate Unit-Price
po['Part-Num'] = po['Part-Num'].str.replace('\t', '') # get rid of \t tab characters in Part-Num

display(po.head(10))
display(purchased.head(10))

,Type,Date,Num,Source-Name,Item,Qty,Cost-Price,Item-Description,Part-Num,PK QTY,Unit-Qty,Unit-Price
0,Purchase Order,2022-11-11,54540,Quality Bearings and Components,7616DLG (INSERT BEARING WITH SHIELDS),3.0,10.00,INSERT BEARING WITH SHIELDS,7616DLG,1.0,3.0,10.00
1,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",R911390100 (BRC MS2N07-E0BNN-BMVG2-NNNNN-NN Servo Motor),5.0,3673.39,BRC MS2N07-E0BNN-BMVG2-NNNNN-NN Servo Motor,R911390100,1.0,5.0,3673.39
2,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",R911384572 (BRC MS2N06-D0BNN-CMSK2-NNNNN-NN Servo Motor),8.0,2414.99,BRC MS2N06-D0BNN-CMSK2-NNNNN-NN Servo Motor,R911384572,1.0,8.0,2414.99
3,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",001283 (BRC Drive package consisting of: R911331611 (HCS01.1E-W0028-A-03-B-ET-EC-NN-L4-NN-FW) an...,8.0,1775.07,BRC Drive package consisting of: R911331611 (HCS01.1E-W0028-A-03-B-ET-EC-NN-L4-NN-FW) and R91134...,001283,1.0,8.0,1775.07
4,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",R911383478 (BRC MS2N03-B0BYN-CMSH0-NNNNN-NN Servo Motor),1.0,1474.15,BRC MS2N03-B0BYN-CMSH0-NNNNN-NN Servo Motor,R911383478,1.0,1.0,1474.15
5,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",001526 (BRC Drive package consisting of: R911397897 (HCS01.1E-W0005-A-03-B-ET-EC-EP-L4-NN-FW) an...,1.0,1527.39,BRC Drive package consisting of: R911397897 (HCS01.1E-W0005-A-03-B-ET-EC-EP-L4-NN-FW) and R91134...,001526,1.0,1.0,1527.39
6,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",R911384424 (BRC MS2N03-B0BYN-CSSK0-NNNNN-NN Servo Motor),0.0,1324.34,BRC MS2N03-B0BYN-CSSK0-NNNNN-NN Servo Motor,R911384424,1.0,0.0,1324.34
7,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",001526 (BRC Drive package consisting of: R911397897 (HCS01.1E-W0005-A-03-B-ET-EC-EP-L4-NN-FW) an...,1.0,1527.39,BRC Drive package consisting of: R911397897 (HCS01.1E-W0005-A-03-B-ET-EC-EP-L4-NN-FW) and R91134...,001526,1.0,1.0,1527.39
8,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",R911390100 (BRC MS2N07-E0BNN-BMVG2-NNNNN-NN Servo Motor),2.0,3673.39,BRC MS2N07-E0BNN-BMVG2-NNNNN-NN Servo Motor,R911390100,1.0,2.0,3673.39
9,Purchase Order,2022-11-16,54563,"AAP Automation, Inc.",001283 (BRC Drive package consisting of: R911331611 (HCS01.1E-W0028-A-03-B-ET-EC-NN-L4-NN-FW) an...,2.0,1775.07,BRC Drive package consisting of: R911331611 (HCS01.1E-W0028-A-03-B-ET-EC-NN-L4-NN-FW) and R91134...,001283,1.0,2.0,1775.07


,Part-Num,Description,PK QTY,Locations
0,HCA-0.5X24.5-277-1850W-O11203,"CUSTOM CARTRIDGE HEATER 24.5"" LONG",1.0,", 2x513.304, 2x523.304,"
1,HCA-0.5X26-277-2000W-O11203,"CUSTOM CARTRIDGE HEATER 26.0"" LONG",1.0,", 2x413.304, 2x423.304, 1x453.304, 1x463.304,"
2,ISO-1/2X26,"0.5” OD Isobar, 26” Long",1.0,", 2x413.303, 2x423.303, 1x453.303, 1x463.303,"
3,ISO-1/2X49,"0.5” OD Isobar, 49” Long",1.0,", 1x513.303, 1x523.303,"
4,MI-ADAPT-BAY-A01,1/8in Bayonet Adapter,1.0,", 6x413.302, 6x423.302, 3x453.302, 3x463.302, 3x513.302, 3x523.302,"
5,SPTC-A100J-3-24-1-AA-O11203,"CUSTOM A100 THERMOCOUPLE 24"" LONG LEADS",1.0,", 2x513.307, 2x523.307,"
6,SPTC-A100J-3-60-1-AA-O11203,"CUSTOM A100 THERMOCOUPLE 60"" LONG LEADS",1.0,", 6x413.306, 6x423.306, 3x453.306, 3x463.306, 1x513.306, 1x523.306,"
7,2301-136Rev.3,Bowl Feeder Assembly per AFS Quote 2301-136Rev.3,1.0,", 1x320.303,"
8,2301-137Rev.3,Bowl Feeder Assembly per AFS Quote 2301-137Rev.3,1.0,", 1x620.303,"
9,ZP-PGA-32-201,"ZIPport panel interface connector, GFCI duplex 120 VAC outlet, (1) RJ45 Ethernet port with female exterior/female interior, 3A circuit breaker",1.0,", 1x001.315,"


In [31]:
# sort po by pk qty
display(po.sort_values(by=['PK QTY'], ascending=False).head(20))
po['Part-Num'].value_counts()[po['Part-Num'].value_counts() > 1].head(50)

,Type,Date,Num,Source-Name,Item,Qty,Cost-Price,Item-Description,Part-Num,PK QTY,Unit-Qty,Unit-Price
691,Purchase Order,2023-08-24,55089,McMaster-Carr Supply Co.,"95610A160 (NYLON PLASTIC WASHER FOR M5 SCREW SIZE, 5.3 mm ID, 11 mm OD, OFF- WHITE, 100PK)",1.0,6.80,"NYLON PLASTIC WASHER FOR M5 SCREW SIZE, 5.3 mm ID, 11 mm OD, OFF- WHITE, 100PK",95610A160,100.0,100.0,0.0680
694,Purchase Order,2023-08-24,55089,McMaster-Carr Supply Co.,"97310A113 (MCM 97310A113 Oversized Washer, M5, 15mm OD)",1.0,6.85,"MCM 97310A113 Oversized Washer, M5, 15mm OD",97310A113,100.0,100.0,0.0685
938,Purchase Order,2023-09-27,55167,McMaster-Carr Supply Co.,"90154A273 (External Retaining Ring for 12 mm OD, Zinc-Chromate-Plated Spring Steel, 100pk)",1.0,11.77,"External Retaining Ring for 12 mm OD, Zinc-Chromate-Plated Spring Steel, 100pk",90154A273,100.0,100.0,0.1177
670,Purchase Order,2023-08-24,55089,McMaster-Carr Supply Co.,"92168A103 (18-8 Stainless Steel Single-Wave Washer for M4 Screw Size, 4.3 mm ID, 9 mm OD, 100pk)",1.0,7.75,"18-8 Stainless Steel Single-Wave Washer for M4 Screw Size, 4.3 mm ID, 9 mm OD, 100pk",92168A103,100.0,100.0,0.0775
622,Purchase Order,2023-08-24,55087,McMaster-Carr Supply Co.,"3088A064 (1008-1010 Carbon Steel Ring Shims 0.5 mm Thick, 8 mm ID, 100pk)",1.0,9.43,"1008-1010 Carbon Steel Ring Shims 0.5 mm Thick, 8 mm ID, 100pk",3088A064,100.0,100.0,0.0943
775,Purchase Order,2023-08-31,55124,McMaster-Carr Supply Co.,"91263A826 (Zinc-Plated Alloy Steel Hex Drive Flat Head Screw M4 x 0.7 mm Thread, 6 mm Long, 100pk)",1.0,8.68,"Zinc-Plated Alloy Steel Hex Drive Flat Head Screw M4 x 0.7 mm Thread, 6 mm Long, 100pk",91263A826,100.0,100.0,0.0868
683,Purchase Order,2023-08-24,55089,McMaster-Carr Supply Co.,"94500A266 (316 Stainless Steel Button Head Hex Drive Screws M3 x 0.5mm Thread, 20mm Long, 100pk)",1.0,7.73,"316 Stainless Steel Button Head Hex Drive Screws M3 x 0.5mm Thread, 20mm Long, 100pk",94500A266,100.0,100.0,0.0773
650,Purchase Order,2023-08-24,55088,McMaster-Carr Supply Co.,"90326A115 (Medium-Strength Steel Thin Hex Nut, Fine-Thread, Zinc-Plated, M10 x 1.25 mm Thread, 1...",1.0,15.75,"Medium-Strength Steel Thin Hex Nut, Fine-Thread, Zinc-Plated, M10 x 1.25 mm Thread, 100pk",90326A115,100.0,100.0,0.1575
680,Purchase Order,2023-08-24,55089,McMaster-Carr Supply Co.,"93625A200 (18-8 Stainless Steel Nylon-Insert Locknut M5 x 0.8 mm Thread, 8 mm Wide, 5 mm High, 1...",1.0,8.77,"18-8 Stainless Steel Nylon-Insert Locknut M5 x 0.8 mm Thread, 8 mm Wide, 5 mm High, 100pk",93625A200,100.0,100.0,0.0877
659,Purchase Order,2023-08-24,55088,McMaster-Carr Supply Co.,"91455A611 (Zinc-Plated†Class†10.9†Steel†Washer,†M6 Screw Size)",1.0,13.79,"Zinc-Plated†Class†10.9†Steel†Washer,†M6 Screw Size",91455A611,100.0,100.0,0.1379


Part-Num
Misc                    84
ANODIZE - CLEAR          8
TBD                      7
Crating                  6
FreightCharge            6
BRC EXPEDITE FEE         5
ANODIZE - HARD CLEAR     5
SCE-9P30LGALV            4
SCE-L9308ELJW            4
5972K91                  3
3842548758               3
94708A213                3
KQ2L08-U02A              3
3842530285               3
001283                   3
91595A579                3
PLATING - CLEAR ZINC     3
JFRM-282835TR12X3        3
3842530283               3
7000-08061-6110750       2
Expedite Charges         2
1AM030AFXK               2
62M060ES423EN            2
1095K23                  2
GF12.111.01.A            2
406936                   2
KQ2H10-U02A              2
BM5-032                  2
90128A262                2
37TP108-17               2
1079N511                 2
5LM012012F               2
R911173397               2
238141-C1                2
D-F7PSAPC                2
303A07500R6              2
34LM012P5          

In [32]:
# merge rows with the same part number in po and add the Qty's together
po[po['Part-Num'] == 'R911372064']

,Type,Date,Num,Source-Name,Item,Qty,Cost-Price,Item-Description,Part-Num,PK QTY,Unit-Qty,Unit-Price
176,Purchase Order,2023-06-15,54940,"AAP Automation, Inc.","R911372064 (BRC RH2-023DBB-NN-010,0)",2.0,475.95,"BRC RH2-023DBB-NN-010,0",R911372064,1.0,2.0,475.95


In [33]:
def parse_locations(locations):
    locations = locations.replace(' ', '') # Remove whitespace
    locations = [location for location in locations.split(',') if location != ''] # Split by comma and remove empty strings (i.e starting and ending commas)
    location_list = []
    # Split apart location and quantity
    for location in locations:
        info = location.split('x')
        info.reverse()
        location_list.append(info)
    return pd.DataFrame(location_list, columns=['Location', 'Qty']).astype({'Qty': 'float'}).round({'Qty': 2})
        

def process_purchased_part(part_num, verbose=False):
    purchased_frame = po[po['Part-Num'] == part_num].copy()
    # sort purchased_frame by date, then by Num
    purchased_frame.sort_values(by=['Date', 'Num'], inplace=True)
    lookup_frame = purchased[purchased['Part-Num'] == part_num].copy()
    purchased_frame.reset_index(drop=True, inplace=True)
    lookup_frame.reset_index(drop=True, inplace=True)

    try:
        locations = parse_locations(lookup_frame['Locations'].iloc[0])
        # locations.loc[0, 'Qty'] = 10
        if verbose:
            display(locations)
    except Exception:
        pass
    # purchased_frame.loc[0, 'Unit-Qty'] = 10
    # duplicate purchased_frame row 1
    # purchased_frame = pd.concat([purchased_frame.iloc[[0]].copy()] * 2).reset_index(drop=True)

    if verbose:
        print("Purchase Order")
        display(purchased_frame)
        print("Purchased Tab")
        display(lookup_frame)
    if purchased_frame.empty: # If part number is not found in PO
        if verbose:
            print('ERROR: Part number not found in PO')
        final_frame = []
        for index, row in locations.iterrows():
            final_frame.append(['<Stock>', np.nan, np.nan, np.nan, lookup_frame['Part-Num'].iloc[0], lookup_frame['Description'].iloc[0], row['Location'], row['Qty'], 0]) 
        final_frame = pd.DataFrame(final_frame, columns=['Type', 'Date', 'Num', 'Source-Name', 'Part-Num', 'Item-Description', 'Location', 'Unit-Qty', 'Unit-Price'])
        return final_frame
    if lookup_frame.empty: # If part number is not found in Purchased
        if verbose:
            print('ERROR: Part number not found in Purchased')
        final_frame = purchased_frame[['Type', 'Date', 'Num', 'Source-Name', 'Part-Num', 'Item-Description', 'Unit-Qty', 'Unit-Price']]
        final_frame.insert(6, 'Location', np.nan)
        final_frame.reset_index(drop=True, inplace=True)
        return final_frame
    else:
        final_frame = []
        for index, row in locations.iterrows():
            # No Qty match
            if purchased_frame.empty:
                final_frame.append(['<Stock>', np.nan, np.nan, np.nan, lookup_frame['Part-Num'].iloc[0], lookup_frame['Description'].iloc[0], row['Location'], row['Qty'], 0])
            # Exact Qty match
            elif row['Qty'] == purchased_frame['Unit-Qty'].iloc[0]:
                if verbose:
                    print('Exact Qty match')
                final_frame.append(purchased_frame[['Type', 'Date', 'Num', 'Source-Name', 'Part-Num', 'Item-Description']].iloc[0].to_list() + [row['Location'], row['Qty'], purchased_frame['Unit-Price'].iloc[0]])
                purchased_frame.drop(0, inplace=True)
                purchased_frame.reset_index(drop=True, inplace=True)
            # Qty in Purchased is less than Qty in PO
            elif row['Qty'] < purchased_frame['Unit-Qty'].iloc[0]:
                if verbose:
                    print('Qty in Purchased is less than Qty in PO')
                final_frame.append(purchased_frame[['Type', 'Date', 'Num', 'Source-Name', 'Part-Num', 'Item-Description']].iloc[0].to_list() + [row['Location'], row['Qty'], purchased_frame['Unit-Price'].iloc[0]])
                purchased_frame.loc[0, 'Unit-Qty'] -= row['Qty']
        # Deal with extra Qty in PO
        if not purchased_frame.empty:
            if verbose:
                print('Qty in Purchased is greater than Qty in PO')
            for index, row in purchased_frame.iterrows():
                final_frame.append(purchased_frame[['Type', 'Date', 'Num', 'Source-Name', 'Part-Num', 'Item-Description']].iloc[index].to_list() + ['<Extra>', row['Unit-Qty'], row['Unit-Price']])
        if verbose:
            display(purchased_frame)
        final_frame = pd.DataFrame(final_frame, columns=['Type', 'Date', 'Num', 'Source-Name', 'Part-Num', 'Item-Description', 'Location', 'Unit-Qty', 'Unit-Price'])
        return final_frame

def get_unique_part_nums():
    po_part_nums = po['Part-Num'].unique()
    purchased_part_nums = purchased['Part-Num'].unique()
    part_nums = np.unique(np.concatenate((po_part_nums, purchased_part_nums), axis=0))
    return part_nums

# process_purchased_part('614NHH', verbose=True)
# process_purchased_part('94708A213')
# process_purchased_part('R911390100', verbose=True)
# process_purchased_part('KQ2L08-U02A', verbose=True)
# process_purchased_part('HCA-0.5X24.5-277-1850W-O11203', verbose=True)
process_purchased_part('1-Up Die for GF12.243.13.B', verbose=True)

Purchase Order


,Type,Date,Num,Source-Name,Item,Qty,Cost-Price,Item-Description,Part-Num,PK QTY,Unit-Qty,Unit-Price
0,Purchase Order,2023-11-29,55317,"Kamei's Dies, Inc.",1-Up Die for GF12.243.13.B (Die for GF12.243.13.B),1.0,120.0,Die for GF12.243.13.B,1-Up Die for GF12.243.13.B,1.0,1.0,120.0


Purchased Tab


,Part-Num,Description,PK QTY,Locations


ERROR: Part number not found in Purchased


,Type,Date,Num,Source-Name,Part-Num,Item-Description,Location,Unit-Qty,Unit-Price
0,Purchase Order,2023-11-29,55317,"Kamei's Dies, Inc.",1-Up Die for GF12.243.13.B,Die for GF12.243.13.B,NaN,1.0,120.0


In [34]:
parts = get_unique_part_nums()
print(parts[:10])
df_list = []
for part in parts:
    print(part)
    df_list.append(process_purchased_part(part))
df = pd.concat([df for df in df_list if not df.empty])
df.reset_index(drop=True, inplace=True)
df['Cost'] = df['Unit-Qty'] * df['Unit-Price']

['001283' '001526' '00221594' '00584951' '00585257' '0120-0410' '01200410'
 '02023-01' '02043842' '022198']
001283
001526
00221594
00584951
00585257
0120-0410
01200410
02023-01
02043842
022198
03220129
06-16-038-0
06-16-038-0 (4.72')
060-16-12-A2
060‐16‐12‐A2
09165546
1-Up Die for GF12.243.13.B
1-Up Die for GF12.342.09.A
1-Up Die for GF12.343.06.B
1-Up Die for GF12.740.03.B
100XL037
1023N13
1023N17
1023N249
1029569
1029591
1043700
1043784
1044072
1044124
1044125
1059450
1075896
1079N511
1085039
1085254
1085343
1095578
1095K23
111010-02
11355T25
1277N714
1277N717
1277N724
1277N725
1283
1321T19
14012
1432N51
1432N65
1445K43
1526
1575A42
1594 CYCLE START
1594 CYCLE STOP
1594-CYCLE START
1594-CYCLE STOP
1594-RESET
159662
159663
159664
159668
159670
159671
159674
159675
159676
16004-A-2Z
1610 1.50
1610 40MM
180XL037
1909A59
1AM030AFXK
1LM030AFXK
201558
2066626
206685
22198
22EDM03-050460B010101
22EDM03-068360C010101
22M6BESD-2828
22MDM03-0252400D010101
22MFES03B-2816
22MFES03C-2816
2301-136

/var/folders/g4/0rdw429j6pz6zgc8bjxh7zqm0000gn/T/ipykernel_46855/1057751858.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df for df in df_list if not df.empty])


In [35]:
# display(df.head(40))
def code_locations(location):
    if location is np.nan:
        return np.nan
    elif location.startswith('001'):
        return 'Main Controls'
    elif location.startswith('1'):
        return 'Main Frame'
    elif location.startswith('2') or location.startswith('002'):
        return 'Unwind/Punch Station'
    elif location.startswith('3') or location.startswith('003') or location.startswith('007'):
        return 'Spout Station'
    elif location.startswith('4') or location.startswith('004'):
        return 'Side Seal Station'
    elif location.startswith('5') or location.startswith('005'):
        return 'Cross Seal Station'
    elif location.startswith('6') or location.startswith('006') or location.startswith('008'):
        return 'Cap Station'
    elif location.startswith('8') or location.startswith('7') or location.startswith('009'):
        return 'Delivery/Cutoff Station'
    return np.nan
df['Cat1'] = df['Location'].apply(code_locations)
# display df where Cat1 is not null and Date is not null
df.to_csv('output/final.csv', index=False)

In [36]:
# show df type value counts
display(df['Type'].value_counts())
display(df['Location'].value_counts())
# number of nulls in location
display(df['Location'].isna().sum())
df.shape

Type
Purchase Order    1804
<Stock>            335
Name: count, dtype: int64

Location
<Extra>    209
513.304      1
423.303      1
413.303      1
160.310      1
          ... 
473.303      1
223.302      1
218.308      1
213.304      1
001.315      1
Name: count, Length: 1288, dtype: int64

643

(2139, 11)

In [14]:
print(len(np.unique(df[df['Location'].isna()]['Part-Num'])))
np.unique(df[df['Location'].isna()]['Part-Num'])

506


array(['001283', '001526', '00221594', '00584951', '00585257', '01200410',
       '02043842', '022198', '03220129', '06-16-038-0', '060-16-12-A2',
       '09165546', '1-Up Die for GF12.243.13.B',
       '1-Up Die for GF12.342.09.A', '1-Up Die for GF12.343.06.B',
       '1-Up Die for GF12.740.03.B', '1095578', '11355T25', '1321T19',
       '14012', '1445K43', '159662', '159663', '159664', '159668',
       '159670', '159671', '159674', '159675', '159676', '2614T27',
       '270003470', '303A07500R6', '303A08MMR6', '303A10000R6',
       '303A12MMR6', '303A14MMR6', '303A20MMR6', '303A35MMR3',
       '303A35MMR6', '3185K68', '3794K511', '37TB-104', '37TB-105',
       '37TB-130', '37TB-136', '37TB-160', '37TB-165', '37TB-78',
       '37TB-80', '37TP108-17', '3842548876', '3842553611', '3842553612',
       '3842553613', '3842555276', '4322K183', '4403K29', '4464K351',
       '4478N21', '450-T18R0', '450-T50R0', '46575A74', '51115K556',
       '51522', '5192N13', '5225K803', '5245K12', '532652